# CAMERA CALIBRATION TUTORIAL

## Learning Objectives 
The Goal for this tutorial will be to help you learn about camera distortions 
that are typically present in photos taken with common pinhole cameras. 
We will also learn the definition and differences between intrinsic vs extrinsic 
parameters of the camera and why they are needed in our code. 
Once these parameters are found, we can use Open CV to undistort the image. 
This is the first step towards full 3D reconstruction.

### The Basics

Today’s cheap pinhole cameras introduces a lot of distortion to images. Two major distortions are radial distortion and tangential distortion.

Due to radial distortion, straight lines will appear curved. Its effect is more as we move away from the center of image. For example, one image is shown below, where two edges of a chess board are marked with red lines. But you can see that border is not a straight line and doesn’t match with the red line. All the expected straight lines are bulged out. Visit Distortion (optics) for more details.

![alt text](../assets/calib_radial.jpg?raw=True "Radial Distortion on checkerboard")

#### Radial Distortion

This distortion is solved as follows:
$$
x_{corrected} = x( 1 + k_1 r^2 + k_2 r^4 + k_3 r^6) \\
y_{corrected} = y( 1 + k_1 r^2 + k_2 r^4 + k_3 r^6)
$$

#### Tangential Distortion
Similarly, another distortion is the tangential distortion which occurs because image taking lense is not aligned perfectly parallel to the imaging plane. So some areas in image may look nearer than expected. It is solved as below:

$$
x_{corrected} = x + [ 2p_1xy + p_2(r^2+2x^2)] \\
y_{corrected} = y + [ p_1(r^2+ 2y^2)+ 2p_2xy]
$$



#### Distortion Coeff
In short, we need to find five parameters, known as distortion coefficients given by:

$$Distortion \; coefficients=(k_1 \hspace{10pt} k_2 \hspace{10pt} p_1 \hspace{10pt} p_2 \hspace{10pt} k_3)$$

#### Camera Matrix

In addition to this, we need to find a few more information, like intrinsic and extrinsic parameters of a camera. Intrinsic parameters are specific to a camera. It includes information like focal length (f_x,f_y), optical centers (c_x, c_y) etc. It is also called camera matrix. It depends on the camera only, so once calculated, it can be stored for future purposes. It is expressed as a 3x3 matrix:

$$camera \; matrix = \left [ \begin{matrix}   f_x & 0 & c_x \\  0 & f_y & c_y \\   0 & 0 & 1 \end{matrix} \right ]$$


Extrinsic parameters corresponds to rotation and translation vectors which translates a coordinates of a 3D point to a coordinate system.

For stereo applications, these distortions need to be corrected first. To find all these parameters, what we have to do is to provide some sample images of a well defined pattern (eg, chess board). We find some specific points in it ( square corners in chess board). We know its coordinates in real world space and we know its coordinates in image. With these data, some mathematical problem is solved in background to get the distortion coefficients. That is the summary of the whole story. For better results, we need atleast 10 test patterns.

## Implementation

For sake of understanding, consider just one image of a chess board. Important input datas needed for camera calibration is a set of 3D real world points and its corresponding 2D image points. 2D image points are OK which we can easily find from the image. (These image points are locations where two black squares touch each other in chess boards)

What about the 3D points from real world space? Those images are taken from a static camera and chess boards are placed at different locations and orientations. So we need to know (X,Y,Z) values. But for simplicity, we can say chess board was kept stationary at XY plane, (so Z=0 always) and camera was moved accordingly. This consideration helps us to find only X,Y values. Now for X,Y values, we can simply pass the points as (0,0), (1,0), (2,0), ... which denotes the location of points. In this case, the results we get will be in the scale of size of chess board square. But if we know the square size, (say 30 mm), and we can pass the values as (0,0),(30,0),(60,0),..., we get the results in mm. (In this case, we don’t know square size since we didn’t take those images, so we pass in terms of square size).

### Packages

In [1]:
import glob # global path/file finder
import cv2
import numpy as np

### Setups

So to find pattern in chess board, we use the function, **cv2.findChessboardCorners()**. We also need to pass what kind of pattern we are looking, like 8x8 grid, 5x5 grid etc. In this example, we use 7x6 grid. (Normally a chess board has 8x8 squares and 7x7 internal corners). It returns the corner points and retval which will be True if pattern is obtained. These corners will be placed in an order (from left-to-right, top-to-bottom)

First, we define a 7 x 6 checkboard corners. It doesn't have to be this shape but for this purpose, we only take 42 points. We use `np.mgrid` to create meshgrid-like points. This object points simplify the representation of 3d point in real world space. In this case, the corners of checker board.

In [15]:
# initialise grid object points of shape (7 x 6)
pshape = (7 * 6, 3)
objp = np.zeros(pshape, np.float32)
objp[:,:2] = np.mgrid[0:7, 0:6].T.reshape(-1, 2)

# array to store object points and image points for all images 
objpoints_list = []  # 3d point in real space 
imgpoints_list = []  # 2d point in image plane

In [4]:
objp[:10].T  # display object point representation for corners

array([[0., 1., 2., 3., 4., 5., 6., 0., 1., 2.],
       [0., 0., 0., 0., 0., 0., 0., 1., 1., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)

In [6]:
# load images
imagepaths = glob.glob("data/*.jpg")

### Finding Corners of Checkerboard

Once we find the corners, we can increase their accuracy using **cv2.cornerSubPix()**. We can also draw the pattern using **cv2.drawChessboardCorners()**. All these steps are included in below code:

In [8]:
# termination criteria for refining corners pix
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

In [16]:
# find chessboard corners for every image in folders './data'
for fname in imagepaths:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # find corner of checker/chess board
    success, corners = cv2.findChessboardCorners(img, (7,6), cv2.CALIB_CB_ADAPpythonTIVE_THRESH)

    if success == True:
        objpoints_list.append(objp)
        
        # refining points of found corners
        corners2  = cv2.cornerSubPix(gray, corners, (11,11), (-1,-1), criteria)
        imgpoints_list.append(corners)

        # draw chessboard corners on image for visualisation
        cv2.drawChessboardCorners(img, (7,6), corners2, success)
        cv2.imshow(fname, img)
        cv2.waitKey(500)

cv2.destroyAllWindows()

### Camera Calibration

In [11]:
img = cv2.imread('data/left11.jpg') # pick an image to demo calibration

In [19]:
height, width, channel = img.shape
success, matrix, distortion, rvecs, tvecs = cv2.calibrateCamera(objpoints_list, 
    imgpoints_list, (width, height), None, None)

### Undistortion

In [21]:
# undistortion
newcameramtx, roi = cv2.getOptimalNewCameraMatrix(matrix, distortion, 
    (width,height), 1, (width,height))
dist = cv2.undistort(img, matrix, distortion, None, newcameramtx)

# crop the image
x, y, w, h = roi
dist = dist[y:y+h, x:x+w]
cv2.imwrite('artifacts/calibresult11.png', dist)

cv2.imshow('calibresult', dist)
cv2.waitKey(500); cv2.destroyAllWindows()

### Saving our results.
We proceed to part II of this tutorial series: Pose Estimation

In [ ]:
# save matrix, distortion coef, rotation, translation
# with np.savez()
outfile = "artifacts/calibresult.npz"
np.savez(outfile, mtx=matrix, distr=distortion, rvecs=rvecs, tvecs=tvecs)

In [22]:
corner = imgpoints_list[0] # sample corners

In [23]:
corner.shape

(42, 1, 2)

In [25]:
corner[0].ravel()

array([160.0524, 306.3398], dtype=float32)